# Introduction to *Blue Book for Bulldozers*

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

/home/jpramos/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jpramos/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jpramos/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
PATH = "data/bulldozers/"

In [4]:
df_train = pd.read_csv(f'{PATH}Train.csv', low_memory=False, 
                     parse_dates=["saledate"])

In [6]:
df_valid = pd.read_csv(f'{PATH}Valid.csv', low_memory=False,
                      parse_dates=["saledate"])

df_vsol = pd.read_csv(f'{PATH}ValidSolution.csv', low_memory=False)
df_vsol.drop(columns=['Usage'], inplace=True)

df_valid = df_valid.merge(df_vsol, left_on='SalesID', right_on='SalesID', how='inner')

In [20]:
df_train.SalePrice = np.log(df_train.SalePrice)
df_valid.SalePrice = np.log(df_valid.SalePrice)

In [7]:
add_datepart(df_train, 'saledate')

add_datepart(df_valid, 'saledate')

In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 65 columns):
SalesID                     401125 non-null int64
SalePrice                   401125 non-null int64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                380989 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    142765 non-null float64
UsageBand                   69639 non-null object
fiModelDesc                 401125 non-null object
fiBaseModel                 401125 non-null object
fiSecondaryDesc             263934 non-null object
fiModelSeries               56908 non-null object
fiModelDescriptor           71919 non-null object
ProductSize                 190350 non-null object
fiProductClassDesc          401125 non-null object
state                       401125 non-null object
ProductGroup                4

In [9]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11573 entries, 0 to 11572
Data columns (total 65 columns):
SalesID                     11573 non-null int64
MachineID                   11573 non-null int64
ModelID                     11573 non-null int64
datasource                  11573 non-null int64
auctioneerID                11573 non-null int64
YearMade                    11573 non-null int64
MachineHoursCurrentMeter    4739 non-null float64
UsageBand                   4031 non-null object
fiModelDesc                 11573 non-null object
fiBaseModel                 11573 non-null object
fiSecondaryDesc             8037 non-null object
fiModelSeries               1759 non-null object
fiModelDescriptor           2897 non-null object
ProductSize                 5743 non-null object
fiProductClassDesc          11573 non-null object
state                       11573 non-null object
ProductGroup                11573 non-null object
ProductGroupDesc            11573 non-null object
Dr

In [10]:
train_cats(df_train)

In [11]:
apply_cats(df_valid, df_train)

In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 65 columns):
SalesID                     401125 non-null int64
SalePrice                   401125 non-null int64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                380989 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    142765 non-null float64
UsageBand                   69639 non-null category
fiModelDesc                 401125 non-null category
fiBaseModel                 401125 non-null category
fiSecondaryDesc             263934 non-null category
fiModelSeries               56908 non-null category
fiModelDescriptor           71919 non-null category
ProductSize                 190350 non-null category
fiProductClassDesc          401125 non-null category
state                       401125 non-null category
ProductGrou

In [13]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11573 entries, 0 to 11572
Data columns (total 65 columns):
SalesID                     11573 non-null int64
MachineID                   11573 non-null int64
ModelID                     11573 non-null int64
datasource                  11573 non-null int64
auctioneerID                11573 non-null int64
YearMade                    11573 non-null int64
MachineHoursCurrentMeter    4739 non-null float64
UsageBand                   4031 non-null category
fiModelDesc                 11485 non-null category
fiBaseModel                 11554 non-null category
fiSecondaryDesc             8035 non-null category
fiModelSeries               1758 non-null category
fiModelDescriptor           2896 non-null category
ProductSize                 5743 non-null category
fiProductClassDesc          11573 non-null category
state                       11573 non-null category
ProductGroup                11573 non-null category
ProductGroupDesc            1157

In [14]:
#??proc_df

In [15]:
X_train, y_train, nas = proc_df(df_train, 'SalePrice')

X_valid, y_valid, _ = proc_df(df_valid, 'SalePrice', na_dict=nas)

In [16]:
df_mt = pd.DataFrame([X_train.UsageBand, df_train.UsageBand])
df_mt

,0,1,2,3,4,5,6,7,8,9,...,401115,401116,401117,401118,401119,401120,401121,401122,401123,401124
UsageBand,2,2,1,1,3,2,1,1,2,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UsageBand,Low,Low,High,High,Medium,Low,High,High,Low,Medium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_mv = pd.DataFrame([X_valid.UsageBand, df_valid.UsageBand])
df_mv

,0,1,2,3,4,5,6,7,8,9,...,11563,11564,11565,11566,11567,11568,11569,11570,11571,11572
UsageBand,0.0,3,3,2,3,0.0,2,2,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UsageBand,NaN,Medium,Medium,Low,Medium,NaN,Low,Low,Low,Low,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)